In [193]:
from blaseball_mike.models import Team, Game
import pandas as pd
import numpy as np

In [32]:
teams_dict = Team.load_all()

In [172]:
team_dfs = []

In [173]:
#get starting pitcher stars for every game
for team, info in teams_dict.items():
    team_name = [info.full_name]
    
    pitchers = [player.name for player in info.rotation] * 20 
    stars = [player.pitching_stars for player in info.rotation] * 20
    
    team_df = pd.DataFrame({'pitcher': pitchers,
                            'pitching_stars': stars,
                            'team': team_name * 100,
                            'day': [i for i in range(0,100)]}).iloc[1:] #offset for pitching rotation
    
    team_dfs.append(team_df)

In [176]:
pitch_schedule = pd.concat(team_dfs)

In [207]:
lineup_dfs = []

In [208]:
for team, info in teams_dict.items():
    team_name = [info.full_name] * 9
    
    batters = [player.name for player in info.lineup]
    stars = [player.batting_stars for player in info.lineup]
    
    team_df = pd.DataFrame({'batter': batters,
                            'batting_stars': stars,
                            'team': team_name})
    
    lineup_dfs.append(team_df)

In [212]:
lineup_stars = pd.concat(lineup_dfs)

mean_bat_stars = lineup_stars.groupby('team').agg('mean').reset_index()

In [170]:
def get_day_df(season, day):
    game = Game.load_by_day(season, day)
    day_df = pd.DataFrame({
                'season':[item.season for item in game.values()],
                'day':[item.day for item in game.values()],
                'game_id': [key for key in game.keys()],
                'home_team': [item.home_team.full_name for item in game.values()],
                'away_team': [item.away_team.full_name for item in game.values()]
                })
    if day % 10 == 0:
        print(day)
    return day_df

def get_regular_season_schedule(season):
    return pd.concat([get_day_df(season, day) for day in range(1, 100)])

In [171]:
season_df = get_regular_season_schedule(7)

10
20
30
40
50
60
70
80
90


,away_team,day,game_id,home_team,season
0,Kansas City Breath Mints,1,1cb778b3-327c-4336-843f-da45de29e85b,Charleston Shoe Thieves,7
1,Dallas Steaks,1,aebbc183-1155-4b06-af15-c2ba28547f6c,Philly Pies,7
2,Mexico City Mild Wings,1,2a9af2d6-dba4-43ee-b9fc-1c4b98c67ca8,Yellowstone Magic,7
3,Baltimore Crabs,1,61aedcd9-4edf-4791-86c7-ffecfed7ed55,San Francisco Lovers,7
4,Boston Flowers,1,3b5c75ab-0787-47c4-a53c-8845d9d218f5,Chicago Firefighters,7


In [200]:
pitch_sched_opp = (pitch_schedule.merge(season_df[['day', 'away_team', 'home_team']],
                                       left_on = ['day', 'team'],
                                       right_on = ['day', 'away_team'],
                                       how = 'left')
                                .merge(season_df[['day', 'away_team', 'home_team']],
                                       left_on = ['day', 'team'],
                                       right_on = ['day', 'home_team'],
                                       how = 'left'))                                    

(1980, 8)

In [202]:
pitch_sched_opp['opp_team'] = np.where(pitch_sched_opp['home_team_x'].isnull()
                                       ,pitch_sched_opp['away_team_y']
                                       ,pitch_sched_opp['home_team_x'])

In [204]:
pitch_sched_opp = pitch_sched_opp[['day', 'pitcher', 'pitching_stars', 'team', 'opp_team']]

In [222]:
matchup_df = pitch_sched_opp.merge(mean_bat_stars,
                                   left_on = 'team',
                                   right_on = 'team',
                                   how = 'left')

,day,pitcher,pitching_stars,team,opp_team,batting_stars
0,1,Stephanie Winters,1.5,Mexico City Mild Wings,Yellowstone Magic,2.111111
1,2,Rafael Davids,3.0,Mexico City Mild Wings,Yellowstone Magic,2.111111
2,3,Burke Gonzales,3.5,Mexico City Mild Wings,Yellowstone Magic,2.111111
3,4,Silvia Rugrat,3.0,Mexico City Mild Wings,Philly Pies,2.111111
4,5,Kennedy Rodgers,1.5,Mexico City Mild Wings,Philly Pies,2.111111


In [223]:
matchup_df['diff'] = matchup_df['pitching_stars'] - matchup_df['batting_stars']

In [226]:
matchup_df = matchup_df.sort_values(by = ['day', 'diff'], ascending = [True, False])

final_rotation = matchup_df.drop_duplicates('day')

In [230]:
final_rotation['pitcher'].value_counts()

Qais Dogwalker        20
Elvis Figueroa        20
Axel Trololol         20
Winnie Hess           20
PolkaDot Patterson    19
Name: pitcher, dtype: int64

In [233]:
final_rotation.to_csv('season_7_rotation.csv')